# Description
This notebook will help setup and run a HAPI FHIR server, and load synthetic data into it.

# Requirements
* Python with the `phdi` package installed
* Java 11+ JDK - not JRE (for Synthea)
* (recommended) wget command line tool

In [2]:
import requests
import os

synthea_url = "https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar"


# Download the synthea jar file
synthea_jar_response = requests.get(synthea_url, stream=True)
if not os.path.isdir("target"):
    os.mkdir("target")

with open("target/synthea-with-dependencies.jar","wb") as file:
    for chunk in synthea_jar_response.iter_content(chunk_size=1024):
        file.write(chunk)